In [ ]:
%cd ..
import os
print(os.getcwd())

In [ ]:
from datetime import date

In [ ]:
from scripts.gtfs_parser import parse_gtfs
from scripts.connectionscan_router import ConnectionScanCore
from scripts.helpers.funs import hhmmss_to_sec, seconds_to_hhmmss
from scripts.helpers.my_logging import init_logging

In [ ]:
init_logging("output", "log.log")

In [ ]:
path_to_gtfs = r"D:\data\90_divers\gtfs (3).zip"

In [ ]:
cs_data = parse_gtfs(path_to_gtfs, date(2019, 8, 1))

In [ ]:
cs_core = ConnectionScanCore(cs_data)

In [ ]:
res = cs_core.route("8507000:0:1", "8509251:0:1", hhmmss_to_sec("07:30:00"))
print(seconds_to_hhmmss(res))

In [ ]:
res = cs_core.route("8507000:0:1", "8509255:0:1", hhmmss_to_sec("07:30:00"))
print(seconds_to_hhmmss(res))

In [ ]:
res = cs_core.route("8590054", "8577003", hhmmss_to_sec("07:30:00"))
print(seconds_to_hhmmss(res) if res else "nichts gefunden")

In [ ]:
res = cs_core.route("8590054", "8509794", hhmmss_to_sec("07:30:00"))
print(seconds_to_hhmmss(res) if res else "nichts gefunden")